In [ ]:
!pip install --upgrade qrisp

Implementing a quantum multiplier (almost) from scratch
-------------------------------------------------------

In this tutorial you will get an insight how Qrisp enables developers to write more and more high-level code starting from a small set of primitive. The goal of the tutorial is to write a function that multiplies two integers. As you might have learned already, this feature is already available via the * operator, so obviously you are not allowed to use it. We don't want to start from absolute zero (ie. gate-level) but restrict ourselves to use only one simple arithmetic primitive:

A simple incrementor.

In [ ]:
from qrisp import *

def increment_by_1(a, inpl_adder = fourier_adder):
    """
    Increments the arguments value by 1, ie. it performs
    a += 1

    Overflow is treated by performing the arithmetic mod 2**len(a).
    
    Parameters
    ----------
    a : list[Qubit] or QuantumVariable
        The quantum value that is incremented.
    inpl_adder : Python callable
        An optional Python function that performs in-place addition. The default is Draper's Fourier adder.
    Returns
    -------
        None.
    """
    inpl_adder(1, a)

To understand what this snippet does, please refer to the [documentation](https://www.qrisp.eu/reference/Miscellaneous%20Functions/generated/qrisp.fourier_adder.html#qrisp.fourier_adder). In simple terms, this function receives an argument $a$, which can either be a list of Qubits or a QuantumVariable, and increments the value by one. For that it calls a customizable adder. For now you can simply ignore the `inpl_adder` keyword - will used it later in the tutorial to build the multiplyer based on an alternative adder.

In [ ]:
a = QuantumFloat(4)
a[:] = 4
print(a)
increment_by_1(a)
print(a)

To move beyond such a simple functionality, we now want to elevate this to a function, that can increment by powers of two. For this, consider how binary addition works:
$$
\begin{array}{cccccccc}
  1 & 0 & 1 & 0 & 1 & 1 & 0 & 1 \\  % First binary number
  + & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\  % Second binary number
&  & & \tiny 1 & &  & &  \\[-2mm]  % Carry line
\hline
  1 & 0 & 1 & 1 & 0 & 1 & 0 & 1 \\  % Result
\end{array}
$$
This instance of long addition adds the number $8 = 2^3 = (1000)_2$ to the number $ 173 = (10101101)_2$. By close examination you might discover that this procedure is basically the same as leaving the lowest 3 bits the same and calling a +1 incrementor on the rest. We can therefore write a function that increments by powers of two without much effort. Hint: QuantumVariables can be turned into lists of Qubits by calling the Python native `list` function on them.

In [ ]:

def increment(u, a, inpl_adder = fourier_adder):
    """
    Increments the quantum value a by the classical value u.
    u has to be a power of 2.

    Parameters
    ----------
    a : list[Qubit] or QuantumVariable
        The quantum value that is incremented.
    u : integer
        A classical integer, which is added to the quantum value. Has to be a power of 2.
    inpl_adder : Python callable
        An optional Python function that performs in-place addition. The default is Draper's Fourier adder.
        
    Returns
    -------
        None.
    """
    if u == 0:
        return
    if (u & (u - 1)):
        raise Exception("Given parameter u is not a power of two")
        
    # TO-DO:
    # Slice the input "a" and call the increment_by_1 function to satisfy the requirements
    # Make sure the inpl_adder keyword is properly transmitted!
    pass

a = QuantumFloat(4)
a[:] = 4
increment(4, a)
print(a)
# Should give {8: 1.0}

In [ ]:
# Test the result
for i in range(7):

    a = QuantumFloat(8)
    b = QuantumFloat(8)
    h(a)
    b[:] = a
    
    increment(2**i, a)

    meas_res = multi_measurement([a,b])

    for a, b in meas_res.keys():

        if (b + 2**i)%(2**8) != a:
            raise
    
print("Congratulations! Your implementation works.")

The next step is to see how multiplication can be understood as a series of controlled additions of powers of two. For this we use the very convenient semi-boolean polynomial formalism that we introduced in one of our [papers](https://arxiv.org/abs/2112.10537). "Semi-boolean" means that the input of the polynomial are booleans, but the coefficients can be integers. The conversion function from the binary representation $(a_0, a_1, a_2 .. a_n)$ to the corresponding integer $a$ can be written as a semi-boolean polynomial:

$$
a = \sum_{i = 0}^{n-1} 2^i a_i
$$

If we have a multiplication of two numbers, the evaluation can also be written as a semi-boolean polynomial:

$$
a \cdot b = \left(\sum_{i = 0}^{n-1} 2^i a_i\right) \cdot \left(\sum_{j = 0}^{n-1} 2^j b_j\right) = \sum_{i,j = 0}^{n-1} 2^{i+j} a_i b_j
$$

In the form of a classical algorithm the last expression therefore looks like this:

In [ ]:
# Create some dummy values for demonstration purposes
a = [False, False, True, True] # Represents the value 1100 = 12
b = [False, True, False, False] # Represents the value 0010 = 2

s = 0
for i in range(len(a)):
    for j in range(len(b)):
        if a[i] and b[j]:
            s += 2**(i+j)
print(s)

Since this is using only incrementations by powers of two, we are already quite close with our function from above! We somehow now need to get the `if` statement into the game. For this we use a Qrisp feature called [ControlEnvironment](https://www.qrisp.eu/reference/Quantum%20Environments/ControlEnvironment.html). This QuantumEnvironment compiles it's code such that it is only executed, if the given control qubits are in the $\ket{1}$ state.

In [ ]:
# Create some test values
a = QuantumFloat(2)
b = QuantumFloat(2)

# Initializes the state |psi> = 1/2**0.5 (|00> + |11>)
h(a[0])
cx(a[0], a[1])

# Performs a controlled incrementation
with control(list(a)):
    increment(2, b)

# Visualizes the statevector
a.qs.statevector()

We see that the incrementation only happened for the state where the control qubits (ie. $a$) were in the $\ket{(11)_2} = \ket{3} $ state.
To set up the multiplier, we will now use the classical code as a blueprint to transfer it to the quantum setting. The first step is declaring the variable $s$. Create a `QuantumFloat` that can hold the result without overflow by considering the bit size of the input $a, b$ using `len(a)`.

In [ ]:
a = QuantumFloat(5)
a[:] = 12

b = QuantumFloat(5)
b[:] = 2


# TO-DO
# Define a QuantumFloat s, that will hold the result of the multiplication without any overflow occuring.


The final step is to fuse everything together: The classical algorithm, together with the incrementor and the knowledge about the ControlEnvironment.

In [ ]:
# TO-DO
# Create the multiplyer
for i in range(len(a)):
    for j in range(len(b)):
        with control([a[i], b[j]]):
            increment(2**(i+j), s)

print(s)

To move up one step in the hierarchy of abstractions, it is not sufficient to simply have some working code. In order to test, maintain, benchmark etc. you need it in the form of a function. Collect the relevant snippets from above to define a function.

In [ ]:

def q_multiplyer(a, b, inpl_adder = fourier_adder):
    """
    Multiplies the values of the QuantumFloats a, b and returns the value as a QuantumFloat
    
    Parameters
    ----------
    a : QuantumFloat
        The first factor.
    a : QuantumFloat
        The second factor.
    Returns
    -------
    s : QuantumFloat
        The product of the two factors.
    """
    
    # TO-DO
    # Collect the code from the previous cells to satisfy the requirements.
    pass

    

In [ ]:
# Test the result
for i in range(1, 5):

    a = QuantumFloat(i)
    b = QuantumFloat(i)
    h(a)
    h(b)
    
    s = q_multiplyer(a, b)

    meas_res = multi_measurement([a,b,s])

    for a, b, s in meas_res.keys():
        if a*b != s:
            raise
    
print("Congratulations! Your implementation works.")

Now that you have a tested solution, you want to benchmark it and subsequently make it more efficient. To measure the performance of your function, you call the `.compile` method of the `.qs` attribute of an arbitrary `QuantumVariable` participating in your solution. The `.qs` attribute contains the [QuantumSession](https://www.qrisp.eu/reference/Core/QuantumSession.html#qrisp.QuantumSession) which manages the lifetime cycle of `QuantumVariable`s and other aspects of high-level abstractions. The [compile](https://www.qrisp.eu/reference/Core/generated/qrisp.QuantumSession.compile.html#qrisp.QuantumSession.compile) method turns the `QuantumSession` into a [QuantumCircuit](https://www.qrisp.eu/reference/Circuit%20Construction/QuantumCircuit.html#qrisp.QuantumCircuit) that allows you to translate your function to Qiskit or other circuit representations.

In [ ]:
a = QuantumFloat(5)
b = QuantumFloat(5)
h(a)
h(b)

s = q_multiplyer(a, b)

qc = s.qs.compile().transpile()

print(f"Circuit depth: {qc.depth()}")
print(f"Qubit count: {qc.num_qubits()}")
print(f"Gate count: {qc.count_ops()}")

Quite good already!

To improve the multiplyer we can now harness the fact we developed the multiplyer in a modular fashion, that is we can easily exchange the adder. The adder that we used so far is described [here](https://arxiv.org/abs/quant-ph/0008033) and utilizes the fact that it is very cheap (in terms of CNOT count) to perform additions on the Fourier transform of the state. The `fourier_adder` function therefore performs a Fourier transform, executes the addition as described in Draper's paper and subsequently reverses the Fourier transform.
Why is this innefficient? Well, we have an algorithm at hand that performs a large number of subsequent additions, so it would be much cheaper to transform once at the beginning and then simply *stay* in the Fourier transform until the end. For situations like this, the `fourier_adder` function exposes the keyword `perform_QFT`, which - if set to `False` skips both Fourier transforms.

In [ ]:
# Define a function that performs fourier addition without entering/leaving fourier space
def reduced_fourier_adder(a, b):
    return fourier_adder(a, b, perform_QFT = False)

a = QuantumFloat(4)
b = QuantumFloat(4)

a[:] = 3
b[:] = 4

# Call the adder wrapped with "manual" fourier transforms
QFT(b, exec_swap = False)

# This performs multiple subsequent adder calls "b += a"
# without peforming a QFT in between.
reduced_fourier_adder(a, b)
reduced_fourier_adder(a, b)
reduced_fourier_adder(a, b)

QFT(b, exec_swap = False, inv = True)

print(b)

Modify the multiplyer by performing the initial and the final QFT manually and call use the `reduced_fourier_adder` for the `increment` function!

In [ ]:

def q_multiplyer(a, b, inpl_adder = fourier_adder):
    """
    Multiplies the values of the QuantumFloats a, b and returns the value as a QuantumFloat
    
    Parameters
    ----------
    a : QuantumFloat
        The first factor.
    a : QuantumFloat
        The second factor.
    Returns
    -------
    s : QuantumFloat
        The product of the two factors.
    """
    
    s = QuantumFloat(len(a) + len(b))

    # TO-DO
    # Modify the code from your q_multiplyer such that the increment function
    # uses the reduced_fourier_adder instead
    
    pass

a = QuantumFloat(5)
a[:] = 5

b = QuantumFloat(5)
b[:] = 7

s = q_multiplyer(a, b)

print(s)


In [ ]:
# Test the result
for i in range(1, 5):

    a = QuantumFloat(i)
    b = QuantumFloat(i)
    h(a)
    h(b)
    
    s = q_multiplyer(a, b)

    meas_res = multi_measurement([a,b,s])
    

    for a, b, s in meas_res.keys():
        if a*b != s:
            raise
    
print("Congratulations! Your implementation works.")

To measure the difference in performance, we benchmark again.

In [ ]:

a = QuantumFloat(5)
b = QuantumFloat(5)
h(a)
h(b)

s = q_multiplyer(a, b)

qc = s.qs.compile().transpile()

print(f"Circuit depth: {qc.depth()}")
print(f"Qubit count: {qc.num_qubits()}")
print(f"Gate count: {qc.count_ops()}")

For the last challenge of this notebook, we want to make you familiar with the `workspace` keyword of the `.compile` method. This keyword allows you to give the compiler some extra qubits to work with, which can reduce the depth. By trying out different options, find the amount of workspace qubits, which is required to bring the depth below 200! Feel free to compare these results to the Qiskit implementation below!

In [ ]:
from qiskit import (QuantumCircuit, QuantumRegister,
ClassicalRegister, Aer, execute, transpile)
from qiskit.circuit.library import RGQFTMultiplier
n = 5

# Create Quantum Registers
a = QuantumRegister(n)
b = QuantumRegister(n)
res = QuantumRegister(2*n)
cl_res = ClassicalRegister(2*n)

# Create QuantumCircuit
qc = QuantumCircuit(a, b, res, cl_res)

# Encode Values 3 and 4
for i in range(len(a)):
    if 3 & 1<<i: qc.x(a[i])
for i in range(len(b)):
    if 4 & 1<<i: qc.x(b[i])

# Load a multiplyer function from the Qiskit library
# and append the gate
qc.append(RGQFTMultiplier(n, 2*n), list(a) + list(b) + list(res))

# Measure the result qubits
qc.measure(res, cl_res)

qc = transpile(qc, basis_gates = ["cx", "u"])

# Perform the simulation
backend = Aer.get_backend('qasm_simulator')
counts_dic = execute(qc, backend).result().get_counts()
print({int(k, 2) : v for k, v in counts_dic.items()})
#Yields: {12: 1024}

# Benchmark
from qiskit import transpile
print(f"Depth: {qc.depth()}")
print(f"Gate count: {qc.count_ops()}")
print(f"Qubit count: {qc.num_qubits}")